In [1]:
import os
from typing import List, Dict, Any
import pandas as pd

In [2]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
)
print("Imports successful")

Imports successful


## Understanding Documents in LangChain

In [12]:
##Understanding Documents in LangChain
doc = Document(
    page_content="This is the content of the document.",
    metadata={"source":"exmaple.txt",
        "pages":1,
        "author":"Prithviraj Acharya",
        "date_created":"2025-01-01",
        "custom_key":"custom_value"
        },
)

print("Document created successfully.")
print(f"Document Source: {doc.page_content}")
print(f"Document metadata: {doc.metadata}")

Document created successfully.
Document Source: This is the content of the document.
Document metadata: {'source': 'exmaple.txt', 'pages': 1, 'author': 'Prithviraj Acharya', 'date_created': '2025-01-01', 'custom_key': 'custom_value'}


In [14]:
## Create a simple txt file
import os
os.makedirs("../data/text_files",exist_ok=True)

In [15]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


## Document Loaders

In [25]:
# Text loader - Read single file

from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/text_files/python_intro.txt")
documents = loader.load()       

print(f"Loaded {len(documents)} document(s).")
print(f"type of document: {type(documents[0])}")
print(f"Document content preview:\n{documents[0].page_content[:200]}...")
print(f"Document metadata:\n{documents[0].metadata}")



Loaded 1 document(s).
type of document: <class 'langchain_core.documents.base.Document'>
Document content preview:
Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has b...
Document metadata:
{'source': '../data/text_files/python_intro.txt'}


In [33]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt", ## Pattern to match files  
    loader_cls= TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=True

)

documents=dir_loader.load()

for i,doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"Content Preview: {doc.page_content[:100]}...")
    print(f"Source: {doc.metadata['source']}")
    print(f"Type: {type(doc)}")
    print(f'Length of document: {len(doc.page_content)} characters')
print(f"\nTotal documents loaded: {len(documents)}")

100%|██████████| 2/2 [00:00<00:00, 2277.66it/s]


Document 1:
Content Preview: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables system...
Source: ..\data\text_files\machine_learning.txt
Type: <class 'langchain_core.documents.base.Document'>
Length of document: 575 characters

Document 2:
Content Preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
Source: ..\data\text_files\python_intro.txt
Type: <class 'langchain_core.documents.base.Document'>
Length of document: 489 characters

Total documents loaded: 2


## Text Splitting